In [1]:
import pathlib
from Utils.wetland_selection    import get_top_scenes
from Utils.treatment_regression import logistic_process
from Utils.outcome_syndata_generator import generate_synthetic_outcome
from Utils.treatment_syndata_generator import generate_synthetic_treatment
from Utils.outcome_regression import regression_process
from Utils.convolution import make_exp_kernel
from Utils.outcome_post import generate_post
import numpy as np

In [2]:
# define folder paths for rasters
DATA_DIR = pathlib.Path('/home/sm79829/Hetwet/Data/Real')    
SYNDATA_DIR = pathlib.Path("/home/sm79829/Hetwet/Data/Synthetic")          

# folders
folders = {
    'wet': DATA_DIR / 'WETLAND_DEV_1996_2016',
    'dem': SYNDATA_DIR / 'DEM',
    'cap': SYNDATA_DIR / 'CAPITAL_1996',
    'claims_96': DATA_DIR / 'LOG_CLAIMS_1996',
    'claims_16': DATA_DIR / 'LOG_CLAIMS_2016',

}

In [3]:
# # top scenes by wetland development
# top_df = get_top_scenes(num_scenes=3566)
# scene_ids = top_df['scene'].astype(str).tolist()

In [3]:
import pandas as pd

# Only select the scenes with valid ids in cap and dem
valid_dem_ids = pd.read_csv(folders['dem'] / "valid_ids_dem.csv").iloc[:, 0].tolist()
valid_cap_ids = pd.read_csv(folders['cap'] / "valid_ids_cap.csv").iloc[:, 0].tolist()

In [4]:
# Intersectino of both valid lists
scene_ids = sorted(set(valid_cap_ids) & set(valid_dem_ids), key=int)
scene_ids = list([str(i) for i in sorted(scene_ids)])
print(f"Number of intersection of valid IDs: {len(scene_ids)}")

Number of intersection of valid IDs: 2195


In [5]:
# estimate treatment logistic regression parameters
theta = 0.5  # classification threshold
logit_res = logistic_process(
    scene_ids=scene_ids,
    folders=folders,
    threshold=theta,
    regularization_C=1.0,
    noise_type='gaussian',
    noise_sd=0.005,
    verbose=False,
    results_dir = "/home/sm79829/Hetwet/Data/Synthetic_2"
)

In [6]:
# generate synthetic treatment
generate_synthetic_treatment(
    scene_ids=scene_ids,
    folders=folders,
    logit_pipe=logit_res,
    threshold=theta,
    noise_sd=0.005,
    noise_type='gaussian',
    results_dir = "/home/sm79829/Hetwet/Data/Synthetic_2"
)


In [7]:
# estimate outcome linear regression parameters
outcome_reg = regression_process(
    scene_ids=scene_ids,
    folders=folders,
    noise_sd=0.1,
    noise_type='gaussian',
    verbose=False,
    results_dir = "/home/sm79829/Hetwet/Data/Synthetic_2"
)

In [8]:
# generate synthetic baseline outcomes:
generate_synthetic_outcome(
    scene_ids = scene_ids,
    folders = folders,
    reg_pipe = outcome_reg,
    target_shape = (8, 8),
    noise_type = "gaussian",
    noise_sd = 0.1,
    results_dir = "/home/sm79829/Hetwet/Data/Synthetic_2"
)

In [9]:
# build your spillover kernel
CELL_SIZE = 30 # cell size in meters
LAMBDA    = 200.0
TRUNCATE = 1
KERNEL    = make_exp_kernel(lam_m=LAMBDA, cell_size_m=CELL_SIZE, truncate=TRUNCATE)
print(f"Number of Affected Nearby Squares: {KERNEL.shape}")

Number of Affected Nearby Squares: (13, 13)


In [10]:
generate_post(
    scene_ids=scene_ids,
    folders=folders,
    KERNEL=KERNEL,
    Bbase = 0.6,   # baseline: 60 % reduction in damages
    Beta1 = 0.07,  # cap: each +1 SD of cap strengthens reduction by ~7 % of baseline
    Beta2 = -0.04, # dem: each +1 SD of dem strengthens reduction by ~4 % of baseline
    noise_sd = 0.1,
    results_dir = "/home/sm79829/Hetwet/Data/Synthetic_2"
)

Using Actual Covariates (No Scaling)


/home/sm79829/.conda/envs/ImageTRT/lib/python3.11/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/sm79829/.conda/envs/ImageTRT/lib/python3.11/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/sm79829/.conda/envs/ImageTRT/lib/python3.11/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/sm79829/.conda/envs/ImageTRT/lib/python3.11/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be 